## Configuration of 5G RAN of Open Air Interface (OAI) with multiple gNBs and multiple UEs
## Objectives
- Configure multiple OAI gNBs and UEs

![multiple_gNB_config_](../FIGS/H05_Multiple_gNB_and_UE/esquema_cenario_multi_gNB_and_multi_UE.drawio.png)


## Linux namespace configuration

To connect multiple gNBs and UEs, you'll need to set up a network namespace in Linux, as well as add route configurations, by following the commands below:

**Step 1:** Creating a namespace:

sudo ip netns add oai-2

**Step 2:** Creating a pair of virtual interfaces (veth):

sudo ip link add veth0 type veth peer name veth1

**Step 3:** Moving one interface to the created namespace:

sudo ip link set veth1 netns oai-2

**Step 4:** Configuring an IP address for veth0 interface:

sudo ip addr add 192.168.1.1/24 dev veth0

**Step 5:** Configuring an IP address for veth1 interface within the namespace:

sudo ip netns exec oai-2 ip addr add 192.168.1.2/24 dev veth1

**Step 6:** Bringing up the veth0 interface:

sudo ip link set veth0 up

**Step 7:** Bringing up the veth1 interface within the namespace:

sudo ip netns exec oai-2 ip link set veth1 up

**Step 8:** Adding a route within the namespace:

sudo ip netns exec oai-2 ip route add 192.168.70.128/26 via 192.168.1.1 dev veth1

**Step 9:** Enabling packet forwarding on the system:

sudo sysctl net.ipv4.conf.all.forwarding=1

**Step 10:** Setting default policy for packet forwarding to ACCEPT:

sudo iptables -P FORWARD ACCEPT

**Step 11:** Enabling packet forwarding within the namespace:

sudo ip netns exec oai-2 sysctl net.ipv4.conf.all.forwarding=1

**Step 12:** Setting default policy for packet forwarding within the namespace to ACCEPT::

sudo ip netns exec oai-2 iptables -P FORWARD ACCEPT

## Multiple gNB Configuration in the monolithic mode

To connect multiple gNBs, follow the steps below in a new Linux Terminal:

**Step 1:** Make a copy of the configuration file used to connect the first gNB: 

cd /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF

sudo cp gnb.sa.band78.fr1.106PRB.usrpb210.conf ./gnb2.sa.band78.fr1.106PRB.usrpb210.conf

**Attention:** You must change the file name, in the example above "gnb2.sa.band78.fr1.106PRB.usrpb210.conf".

**Step 2:**  Change gNB parameters

This part you will change the gNB name and ID in (/home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb2.sa.band78.fr1.106PRB.usrpb210.conf), such as in figure below.

![multiple_gNB_config_](../FIGS/H02_5GRAN_UNI_III/multiple_gNB_config.png)

**Step 3:** Chenge gNB network interfaces:

In the same file as in Step 2, change the network interfaces according to the namespace you created earlier, such as in figure below.

![multiple_gNB_config_](../FIGS/H05_Multiple_gNB_and_UE/multiple_gnb_2.png)

Open file by typing:

gedit /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb2.sa.band78.fr1.106PRB.usrpb210.conf

**Attention:** The name in "Active_gNBs" has to be the same as the parameter "gNB_name", in the example of the figure above "gNB2-OAI".

**Attention:** To connect other gNBs, simply repeat the steps, always entering different names and IDs for each gNB.

## Multiple gNB and UE deployment

To connect two UEs, one in each gNB, it will be necessary to deploy each "UE <-> gNB" pair separately, with one UE and its respective gNB inside the namespace created in the previous step, and the other pair outside the namespace.

**Step 1:** Deployment of first UE and first gNB inside the namespace oai-2:

***Step 1.1:*** Deployment of gNB:

First, in a Linux terminal, go into the namespace you created:

sudo ip netns exec oai-2 bash

Next, to deploy the gNB, type in terminal:

cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build/

sudo RFSIMULATOR=server ./nr-softmodem --rfsim --sa -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb2.sa.band78.fr1.106PRB.usrpb210.conf --gNBs.[0].min_rxtxtime 6 --continuous-tx

***Step 1.2:*** Deployment of UE:

First, in another Linux terminal, go into the namespace you created:

sudo ip netns exec oai-2 bash

Next, to deploy the UE, type in terminal:

cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build/

sudo RFSIMULATOR=192.168.1.2 ./nr-uesoftmodem -r 106 --numerology 1 --band 78 -C 3619200000 --rfsim --sa --nokrnmod -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf --uicc0.imsi 001010000000001

**Attention:** You must execute the command above with the "RFSIMULATOR" parameter with the IP used in the gNB interface configuration step  

**Step 2:** Deployment of second UE and second gNB outside the namespace oai-2:

***Step 2.1:*** Deployment of gNB:

To deploy the gNB, type in a new Linux terminal:

cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build/

sudo RFSIMULATOR=server ./nr-softmodem --rfsim --sa -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf --gNBs.[0].min_rxtxtime 6 --continuous-tx

***Step 2.2:*** Deployment of UE:

To deploy the UE, type in a new Linux terminal:

cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build/

sudo RFSIMULATOR=192.168.70.129 ./nr-uesoftmodem -r 106 --numerology 1 --band 78 -C 3619200000 --rfsim --sa --nokrnmod -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf --uicc0.imsi 001010000000002

**Attention:** Note that in the deployment command, the parameter "--uicc0.imsi 001010000000002" has been added, referring to the IMSI of the second UE, you MUST enter an IMSI that is present in the database (as shown in the "UE Configuration" section) and that IS NOT BEING USED BY ANOTHER UE.